In [1]:
#import packages
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/u/yashjain/anaconda3/envs/vitessce-env/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#set directories for input and output
print(os.getcwd())

/u/yashjain/kaggle_4/paper-analysis-code/competition-timeline-analysis


In [3]:

IN_DIR = os.getcwd()
OUT_DIR = os.getcwd() + "/filtered"

# IN_DIR = "/Users/adarsh_23/CNS/Envisioning-Kaggle-main/Metadata"
# OUT_DIR = "/Users/adarsh_23/CNS/Envisioning-Kaggle-main/Filtered"


In [4]:
#read in dfs and declare variables to select competition of interest
comps_df = pd.read_csv(IN_DIR+'/Competitions.csv', parse_dates=['DeadlineDate'])
#comps_df.shape
comp_title = "SenNet + HOA - Hacking the Human Vasculature in 3D"
comp_overview = comps_df[comps_df.Title==comp_title]
comp_id = comp_overview.iloc[0]['Id']
#comp_id
comp_forum_id = comp_overview.iloc[0]['ForumId']
comp_forum_id

3859014

In [5]:
comp_overview

,Id,Slug,Title,Subtitle,HostSegmentTitle,ForumId,OrganizationId,EnabledDate,DeadlineDate,ProhibitNewEntrantsDeadlineDate,...,CanQualifyTiers,TotalTeams,TotalCompetitors,TotalSubmissions,ValidationSetName,ValidationSetValue,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,HostName,CompetitionTypeId
5678,61446,blood-vessel-segmentation,SenNet + HOA - Hacking the Human Vasculature i...,Segment vasculature in 3D scans of human kidney,Research,3859014,3873.0,11/07/2023 17:26:14,2024-02-06 23:59:00,01/30/2024 23:59:00,...,True,1149,1401,32391,NaN,NaN,False,False,NaN,1


In [6]:
#write filtered competitions dataframe to csv
comp_overview.to_csv(OUT_DIR+'/Competitions.csv')

In [7]:
# filter and write Teams
teams_df = pd.read_csv(IN_DIR+'/Teams.csv', infer_datetime_format=True)

comp_teams_df = teams_df[teams_df.CompetitionId==comp_id].dropna(subset=['LastSubmissionDate'])

team_ids_list = comp_teams_df.Id.unique().tolist()
#len(team_ids_list)

comp_teams_df.to_csv(OUT_DIR+'/Teams.csv')

In [8]:
#filter forums, forumtopics, forumMessages, and forum Message Votes
forums_df = pd.read_csv(IN_DIR+'/Forums.csv', infer_datetime_format=True)
forums_topics_df = pd.read_csv(IN_DIR+'/ForumTopics.csv', infer_datetime_format=True)
forums_messages_df = pd.read_csv(IN_DIR+'/ForumMessages.csv', infer_datetime_format=True)
forums_message_votes_df = pd.read_csv(IN_DIR+'/ForumMessageVotes.csv', infer_datetime_format=True)

In [9]:
comp_forums_df = forums_df[forums_df.Id==comp_forum_id]
comp_forums_topics_df = forums_topics_df[forums_topics_df.ForumId==comp_forum_id]

#have to filter by forum topic
topic_ids = list(comp_forums_topics_df.Id.unique())
#topic_ids

comp_forums_messages_df = forums_messages_df[forums_messages_df.ForumTopicId.isin(topic_ids)]
#comp_forums_messages_df.head(5)

#have to filter by forum message
message_ids = list(comp_forums_messages_df.Id.unique())
comp_forums_message_votes_df = forums_message_votes_df[forums_message_votes_df.ForumMessageId.isin(message_ids)]
comp_forums_message_votes_df.head(5)

,Id,ForumMessageId,FromUserId,ToUserId,VoteDate
2908967,3782721,2511311,3012786,1536542,11/07/2023
2908968,3782749,2511311,16658843,1536542,11/07/2023
2908969,3784305,2511311,5122276,1536542,11/08/2023
2908970,3784534,2511311,2959458,1536542,11/08/2023
2908971,3786857,2511311,6261540,1536542,11/09/2023


In [10]:
comp_forums_messages_df.shape


(1599, 9)

In [11]:
comp_forums_df.to_csv(OUT_DIR+'/Forums.csv')
comp_forums_topics_df.to_csv(OUT_DIR+'/ForumTopics.csv')
comp_forums_messages_df.to_csv(OUT_DIR+'/ForumMessages.csv')
comp_forums_message_votes_df.to_csv(OUT_DIR+'/ForumMessageVotes.csv')

In [12]:
#Filter users by whether they are on a team in the competition or not
users_df = pd.read_csv(IN_DIR+'/Users.csv', infer_datetime_format=True)
#for this, I will also need the teammemberships info
team_memberships_df = pd.read_csv(IN_DIR+'/TeamMemberships.csv', infer_datetime_format=True)
team_memberships_df.shape


(7701819, 4)

In [13]:
comp_team_memberships_df = team_memberships_df[team_memberships_df.TeamId.isin(team_ids_list)]
comp_team_memberships_df.head(5)

,Id,TeamId,UserId,RequestDate
7200699,11523087,11205901,2917570,11/07/2023
7200702,11523158,11205974,2228534,11/07/2023
7200710,11739988,11206255,6388,12/19/2023
7200720,11523740,11206545,13373958,11/07/2023
7200738,11524437,11207209,6280504,11/07/2023


In [14]:
comp_users_list = list(comp_team_memberships_df.UserId.unique())
#comp_users_list
comp_users_df = users_df[users_df.Id.isin(comp_users_list)]
comp_users_df.to_csv(OUT_DIR+'/Users.csv')


In [15]:
kernelVersionCompSources_df = pd.read_csv(IN_DIR+'/KernelVersionCompetitionSources.csv', infer_datetime_format=True)

In [16]:
comp_kernelversionids_df = kernelVersionCompSources_df[kernelVersionCompSources_df.SourceCompetitionId==comp_id]
comp_kernelversionids_df.shape

(4741, 3)

In [17]:
comp_kernel_ids_list = comp_kernelversionids_df.KernelVersionId.unique()
len(comp_kernel_ids_list)

4741

In [18]:
kernels_df = pd.read_csv(IN_DIR+'/Kernels.csv', infer_datetime_format=True)
kernels_df.shape

(1216724, 16)

In [19]:
comp_kernels_df = kernels_df[kernels_df.CurrentKernelVersionId.isin(comp_kernel_ids_list)]
#comp_kernels_df.shape (350,16)
comp_kernels_df.head()

,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
1010386,45825732,11013012,149821771.0,NaN,453819.0,149755499.0,11/07/2023 19:28:14,11/08/2023,11/07/2023,False,sennet-hoa-visualize-3d-slice-mask,2.0,11/09/2023,2554,8,41
1010459,45830063,3012786,149781722.0,NaN,453902.0,149766668.0,11/07/2023 21:42:23,11/08/2023,11/08/2023,False,kidney-senescence-pyvips,3.0,11/08/2023,511,4,16
1010500,45835069,11740424,149945203.0,NaN,NaN,149777616.0,11/08/2023 00:29:00,11/09/2023,11/09/2023,False,preparing-coco-dataset,NaN,NaN,173,0,3
1010505,45835500,4199239,149997576.0,NaN,NaN,149778564.0,11/08/2023 00:44:11,11/09/2023,11/08/2023,False,blood-vessel-segmentation,NaN,06/04/2024,408,0,5
1010556,45840016,9240143,150604837.0,NaN,453988.0,149788604.0,11/08/2023 02:40:00,11/14/2023,11/08/2023,False,tf-keras-unet-starter-sennet-hoa,3.0,11/09/2023,1249,10,20


In [20]:
comp_kernels_df.shape

(211, 16)

In [21]:
comp_kernels_df.to_csv(OUT_DIR+'/Kernels.csv')


In [22]:
comp_kernels_df.shape


(211, 16)

In [23]:
#filter datasets by forum id: 995088.0

datasets_df = pd.read_csv(IN_DIR+'/Datasets.csv')
print(datasets_df.shape) #(183770, 14)
datasets_df.head()

(375979, 14)


,Id,CreatorUserId,OwnerUserId,OwnerOrganizationId,CurrentDatasetVersionId,CurrentDatasourceVersionId,ForumId,Type,CreationDate,LastActivityDate,TotalViews,TotalDownloads,TotalVotes,TotalKernels
0,7,491632,NaN,4.0,469900.0,485806.0,766,Dataset,08/04/2015 23:59:00,02/06/2018,118705,10128,287,600
1,8,495305,NaN,6.0,7142.0,7142.0,774,Dataset,08/18/2015 21:53:00,01/31/2018,49302,4188,113,77
2,21,1,NaN,9.0,2159.0,2159.0,1015,Dataset,01/20/2016 16:32:33,02/05/2018,353770,31266,683,198
3,22,508688,NaN,4.0,2152.0,2152.0,1016,Dataset,01/20/2016 22:51:48,01/17/2018,18536,1197,51,20
4,39,583372,583372.0,NaN,800033.0,821952.0,1285,Dataset,05/16/2016 21:44:11,02/01/2018,32858,1669,40,18


In [24]:
comp_datasets_df = datasets_df[datasets_df.ForumId==int(comp_forum_id)]
comp_datasets_df.shape

(0, 14)

In [25]:
int(comp_forum_id)

3859014

In [26]:
print(datasets_df.ForumId.unique())

[    766     774    1015 ... 5733163 5733358 5733494]


In [27]:
# troubleshooting kernal lineage network
kernels_df.head()


,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
0,1,2505,205.0,NaN,NaN,1.0,03/25/2015 18:25:32,03/23/2018,03/25/2015,False,hello,NaN,NaN,392,0,0
1,2,3716,1748.0,NaN,26670.0,2.0,03/25/2015 18:31:07,04/16/2015,03/25/2015,False,rf-proximity,3.0,07/15/2016,9147,1,13
2,4,3716,41.0,NaN,NaN,9.0,03/25/2015 21:57:36,03/23/2018,03/25/2015,False,r-version,NaN,NaN,167,0,0
3,5,28963,19.0,NaN,NaN,13.0,03/25/2015 22:01:04,03/23/2018,03/25/2015,False,test1,NaN,NaN,143,0,0
4,6,3716,21.0,NaN,NaN,15.0,03/25/2015 22:19:00,03/23/2018,03/25/2015,False,are-icons-missing,NaN,NaN,164,0,0


In [28]:
comp_kernels_df.head()

,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
1010386,45825732,11013012,149821771.0,NaN,453819.0,149755499.0,11/07/2023 19:28:14,11/08/2023,11/07/2023,False,sennet-hoa-visualize-3d-slice-mask,2.0,11/09/2023,2554,8,41
1010459,45830063,3012786,149781722.0,NaN,453902.0,149766668.0,11/07/2023 21:42:23,11/08/2023,11/08/2023,False,kidney-senescence-pyvips,3.0,11/08/2023,511,4,16
1010500,45835069,11740424,149945203.0,NaN,NaN,149777616.0,11/08/2023 00:29:00,11/09/2023,11/09/2023,False,preparing-coco-dataset,NaN,NaN,173,0,3
1010505,45835500,4199239,149997576.0,NaN,NaN,149778564.0,11/08/2023 00:44:11,11/09/2023,11/08/2023,False,blood-vessel-segmentation,NaN,06/04/2024,408,0,5
1010556,45840016,9240143,150604837.0,NaN,453988.0,149788604.0,11/08/2023 02:40:00,11/14/2023,11/08/2023,False,tf-keras-unet-starter-sennet-hoa,3.0,11/09/2023,1249,10,20


In [29]:
# target = comp_kernels_df[comp_kernels_df.CurrentUrlSlug=="unet-baseline-train"]
# target

,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes


In [30]:
# target_parent = kernels_df[kernels_df.CurrentKernelVersionId==47074200]
# target_parent


,Id,AuthorUserId,CurrentKernelVersionId,ForkParentKernelVersionId,ForumTopicId,FirstKernelVersionId,CreationDate,EvaluationDate,MadePublicDate,IsProjectLanguageTemplate,CurrentUrlSlug,Medal,MedalAwardDate,TotalViews,TotalComments,TotalVotes
430480,12930165,1094066,47074200.0,NaN,NaN,47065816.0,11/17/2020 08:28:26,11/17/2020,11/17/2020,False,hubmap-res34unet-baseline-train,2.0,11/29/2020,4773,0,30


In [30]:
#filter submissions
submissions_df = pd.read_csv(IN_DIR+'/Submissions.csv', infer_datetime_format=True)
submissions_df

,Id,SubmittedUserId,TeamId,SourceKernelVersionId,SubmissionDate,ScoreDate,IsAfterDeadline,PublicScoreLeaderboardDisplay,PublicScoreFullPrecision,PrivateScoreLeaderboardDisplay,PrivateScoreFullPrecision
0,271144,40620.0,35790,NaN,03/08/2013,NaN,True,0.90246,0.902464,0.91024,0.910241
1,269508,40620.0,35790,NaN,03/04/2013,NaN,True,0.90101,0.901012,0.88353,0.883531
2,269504,40620.0,35790,NaN,03/04/2013,NaN,True,0.89230,0.892306,0.87690,0.876903
3,269497,40620.0,35790,NaN,03/04/2013,NaN,True,0.89294,0.892944,0.88089,0.880899
4,269177,40620.0,35790,NaN,03/03/2013,NaN,True,0.78117,0.781174,0.76231,0.762319
...,...,...,...,...,...,...,...,...,...,...,...
14577627,39318811,21698254.0,12432897,NaN,07/27/2024,07/27/2024,True,0.68029,0.680298,0.68029,0.680298
14577628,39527278,20218683.0,12492986,NaN,08/13/2024,08/13/2024,True,0.81614,0.816143,0.82917,0.829175
14577629,39527258,20218683.0,12492986,NaN,08/13/2024,08/13/2024,True,0.88340,0.883408,0.88867,0.888676
14577630,39527177,20218683.0,12492986,NaN,08/13/2024,08/13/2024,True,0.84753,0.847534,0.86564,0.865643


In [31]:
comp_submissions_df = submissions_df[submissions_df.TeamId.isin(team_ids_list)]
comp_submissions_df

,Id,SubmittedUserId,TeamId,SourceKernelVersionId,SubmissionDate,ScoreDate,IsAfterDeadline,PublicScoreLeaderboardDisplay,PublicScoreFullPrecision,PrivateScoreLeaderboardDisplay,PrivateScoreFullPrecision
13330517,34824672,9240143.0,11208527,151877948.0,11/22/2023,11/22/2023,False,0.000000,0.000000,0.000000,0.000000
13330518,34673975,9240143.0,11208527,151007289.0,11/16/2023,11/16/2023,False,0.000000,0.000000,0.000000,0.000000
13330519,34638005,9240143.0,11208527,150746669.0,11/15/2023,11/15/2023,False,0.000000,0.000000,0.000000,0.000000
13330520,34833580,9240143.0,11208527,151920526.0,11/23/2023,11/23/2023,False,0.000000,0.000000,0.000000,0.000000
13330521,34616893,9240143.0,11208527,150604837.0,11/14/2023,11/14/2023,False,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
14549643,39203060,6625861.0,11209445,188790276.0,07/18/2024,07/18/2024,True,NaN,NaN,NaN,NaN
14549644,39202942,6625861.0,11209445,188788797.0,07/18/2024,07/18/2024,True,NaN,NaN,NaN,NaN
14549645,39207494,6625861.0,11209445,188837538.0,07/18/2024,07/18/2024,True,0.670762,0.670762,0.723641,0.723641
14555791,39511238,9253999.0,11487688,192285976.0,08/12/2024,08/12/2024,True,NaN,NaN,NaN,NaN


In [32]:
comp_submissions_df.to_csv(OUT_DIR+'/Submissions.csv')
